# The Relation between the SEIR Epidemic Model and Hawkes Processes
> Recently, a link between the SIR compartmental epidemic model and Hawkes-like processes was found. This opens up the possibility to use some of the rich theory related to Hawkes processes when analyzing spreads of diseases using the SIR model. As the SIR model is not capable of modeling incubation times – a feature inherent in Covid-19 and many other diseases – there is a need to extend the recently found link to more complex compartmental epidemic models like the SEIR model. We have discovered this link and present it briefly in this blog post. For a detailed derivation of our results and a more in-depth discussion please refer to <a href="https://todo">our paper</a>.
- toc: false
- branch: master
- badges: true
- comments: true
- categories: [epidemic models]
- hide: false
- search_exclude: true

In [1]:
#hide
import altair as alt

In [2]:
#hide
from sim_params import *

In [3]:
#hide
slider_beta = alt.binding_range(min=beta_min, max=beta_max, step=beta_step, name="\u03b2 ")
slider_sigma = alt.binding_range(min=gamma_min, max=gamma_max, step=gamma_step, name="\u03c3 ")
slider_gamma = alt.binding_range(min=gamma_min, max=gamma_max, step=gamma_step, name="\u03b3 ")

select_beta = alt.selection_single(
    fields=["beta"],
    bind={"beta": slider_beta},
    init={"beta": 1},
    name="parameter_beta"
)

select_gamma = alt.selection_single(
    fields=["gamma"],
    bind={"gamma": slider_gamma},
    init={"gamma": .1},
    name="parameter_gamma"
)

select_sigma = alt.selection_single(
    fields=["sigma"],
    bind={"sigma": slider_sigma},
    init={"sigma": .3},
    name="parameter_sigma"
)

In [4]:
#hide
sir_data = {"source": "https://raw.githubusercontent.com/yogabonito/test/master/_notebooks/plots/sir.json", "title": "SIR"}
seir_data = {"source": "https://raw.githubusercontent.com/yogabonito/test/master/_notebooks/plots/seir.json", "title": "SEIR"}
exp_hawkesn_data = {
    "source": "https://raw.githubusercontent.com/yogabonito/test/master/_notebooks/plots/exp_hawkesn.json",
    "title": "HawkesN process with exponential kernel"
}
hypoexp_and_erlang_hawkesn_data = {
    "source": "https://raw.githubusercontent.com/yogabonito/test/master/_notebooks/plots/hypoexp_and_erlang_hawkes.json",
    "title": "HawkesN process with hypoexponential/Erlang kernel"
}

sim_data = [sir_data, seir_data, exp_hawkesn_data, hypoexp_and_erlang_hawkesn_data]

In [44]:
#hide
sir_chart, seir_chart, exp_hawkesn_chart, hypoexp_and_erlang_hawkesn_chart = (
    alt.Chart(data["source"]).mark_bar().encode(
        alt.X(
            "day:Q",
            scale=alt.Scale(domain=(1, t_max-5))
        ),    
        alt.Y(
            "cumulative infections:Q",
            scale=alt.Scale(domain=(0, n))
        ),
        #x="day:Q"
        tooltip=["day:Q", "cumulative infections:Q"]
    ).properties(
        title=data["title"]
    )
    for data in sim_data
)

sir_chart, seir_chart, exp_hawkesn_chart, hypoexp_and_erlang_hawkesn_chart = (
    chart.add_selection(
        select_gamma
    ).transform_filter(
        select_gamma
    ).add_selection(
        select_beta
    ).transform_filter(
        select_beta
    )
    for chart in [sir_chart, seir_chart, exp_hawkesn_chart, hypoexp_and_erlang_hawkesn_chart]
)

seir_chart, hypoexp_and_erlang_hawkesn_chart = (
    chart.add_selection(
        select_sigma
    ).transform_filter(
        select_sigma
    )
    for chart in [seir_chart, hypoexp_and_erlang_hawkesn_chart]
)

In [45]:
#hide_input
(sir_chart | seir_chart) & (exp_hawkesn_chart | hypoexp_and_erlang_hawkesn_chart)

alt.VConcatChart(...)

## Introduction

Since their introduction in the 1970s, Hawkes processes have been successfully applied in many different fields, such as seismology, finance, and information science. Alan G. Hawkes also considered his class of stochastic processes "a possible epidemic model in large populations"{% fn 1 %}.

However, a model assuming an infinitely large population is often not suitable when modeling the spread of diseases. That's why many researchers preferred compartmental epidemic models (like the SIR or SEIR model) to Hawkes processes.

Only recently, it has been shown that there is a link between Hawkes-like processes and the SIR compartmental model.{% fn 2 %} As Rizoiu et al. put it, "[t]his is significant, as it
indicates that tools developed for one approach can be applied to the other." However, this link is restricted to the SIR compartmental model. As this model lacks the capability to model incubation time, many researchers turn to more complex models such as the SEIR model and variants thereof. This is especially true for diffusion analyses in the current Covid-19 crisis.

We have discovered a link between the SEIR model and Hawkes-like processes.{% fn 3 %} In this blog post, we explain it briefly and give an overview of all the models mentioned so far.

But before we dive into the theory, let's get a feel of the models we will talk about. The plot at the beginning of this blog post shows four subplots, each containing the total number of infections (y-axis) that occured until a certain day (x-axis) in a population of 10 thousand people using one specific simulation model. When you hover over the bars, you will notice that many infection counts are not integers; that's because they are calculated as the mean of many simulations. Fiddle around with the sliders below the plot and see how they affect the spread of a disease. Comparing the subplots, we notice three things:

> Note: The two subplots on the left look similar. This is a consequence of the finding of Rizoiu et al. that we mentioned above: There is a link between the SIR model and Hawkes-like processes.

> Note: The two subplots on the right look similar. This is a consequence of our finding: There is a link between the SEIR model and Hawkes-like processes.

> Important: The subplots on the left and on the right are **very** different! Also note, that you can adjust the latency in the models plotted on the right (higher values for $\sigma$ implying less latency). The models on the left, however, are not capable of modeling latency (and thus incubation time). This underlines the importance of extending the link found by Rizoiu et al. to other compartmental models.

## Hawkes Processes

A Hawkes process is a type of counting process. A counting process counts events. These events can for example be infections (in epidemiology) or earthquakes (in seismology). Whenever an event occurs, the counting process experiences a jump of size $1$. Let's have a look at a sample path of a Hawkes process $N_t$ in the following figure (the blue line).

![Hawkes process - sample path and corresponding conditional intensity](./fig/hawkes_sample_path_with_conditional_intensity.svg)

You will notice that its jumps are not evenly distributed over the observed time interval $[0, 60]$. Why's that?

It's because the behavior of $N_t$ is dictated by its so-called conditional intensity $\lambda_t$ (the orange line). Whenever $\lambda_t$ is high we have a high probability of events. A low $\lambda_t$, on the other hand, leads to fewer jumps in $N_t$. This property is valid for all counting processes.

What makes Hawkes processes so special is that their conditional intensity $\lambda_t$ again depends on the counting process. In the figure above we see that the jumps in $N_t$ also increase the conditional intensity.

In general, the conditional intensity of a Hawkes process has the form
\begin{align}
\lambda_t
= \mu(t) + \sum\limits_{t_j < t} \nu(t-t_j)
,
\end{align}
where $t_j$ is an event time and $\mu(t)$ as well as $\nu(t-t_j)$ are both $\geq 0$.{% fn 4 %}
We call $\mu(t)$ the *background intensity*. It is independent of $N_t$.

$\nu(t - t_j)$, on the other hand, depends on $t_j$ and thus on the history of the counting process $N_t$. We call $\nu$ the *kernel* of the Hawkes process.

In the figure above, $\mu(t) = \mu$ was constant and $\nu$ was an exponential kernel, this means
\begin{align}
\lambda_t
= \mu
+
\sum\limits_{t_j < t} \kappa \theta \text{e}^{-\theta (t - t_j)}.
\end{align}

Here, $\kappa$ is called the *scale* parameter and $\theta$ is called the *decay* parameter. You may notice that this kernel equals the density of the exponential distribution with parameter $\theta$ scaled by the parameter $\kappa$.

What is important is that the exponential kernel leads to immediate jumps in $\lambda_t$ when an event occurs. When modeling infections, this means that an infection immediately increases the probability of further infections. This also means, that there is no way of taking incubation time into account. We will adress this issue below. But first, let's head to the other class of models we discuss in this post.

## Compartmental Epidemic Models

## The SIR Model

### Overview

Compartmental epidemic models (such as SIR and SEIR) model the spread of a disease within some population of size $N$ by dividing this population into groups (so-called compartments). In the SIR model these groups are called
 - S (group of all *susceptibles*),
 - I (group of all *infected*), and
 - R (group of all *recovered*).

Susceptibles are healthy. After becoming infected, they move to compartment I. After some time infected people recover from a disease and hence move to compartment R. The possible transitions between compartments are outlined in the following figure.

![SIR model](./fig/flowcharts/sir.svg)

The exact laws that govern the rates at wich people transition from one compartment to the next can be expressed in various ways. In the so-called CTMC-SIR model this is achieved via stochastic differential equations. For brevity (and as they are not our main focus), we will not present them here.

Let $S_t$ be the number of people in compartment S at time $t$. We define $I_t$ and $R_t$ analogously. Now $S_t$, $I_t$, and $R_t$ are stochastic processes. Now we can establish a link between the SIR model and Hawkes processes by finding similarities in conditional intensities.

### The link to Hawkes processes

How does a conditional intensity look like in the SIR model? This depends on which process in this model we are considering. As we are interested in infections we might be tempted to choose the process $I_t$. However, $I_t$ is not a counting process, so we can't compute a conditional intensity. (A counting process counts and thus, it is monotonically increasing, but $I_t$ decreases when people transition from I to R.)

We can solve this issue by choosing the process $I_t + R_t$. This represents a counting process, which counts all infections that have occurred until time $t$. Now it is possible to derive the conditional intensity $\lambda_t^{SIR}$ from the stochastic differential equations of the SIR model.

Unfortunately, the result is not directly comparable to the conditional intensity of a Hawkes process. This is because a Hawkes process can only model infections, whereas the SIR model also contains recoveries as events. Therefore, Rizoiu et al. calculated an expected value, integrating over the recovery times, which are not observable in the Hawkes process. If we call the set of all recovery times $\mathcal{T}$, then the main result of Rizoiu et al. is
\begin{align}
\mathbb{E}_{\mathcal{T}} \left(  \lambda_t^{SIR} \right)
= \left( 1 - \frac{I_t + R_t}{N} \right)
\sum\limits_{t_j < t} \beta \text{e}^{-\gamma (t - t_j)},
\end{align}
where $\beta$ and $\gamma$ are the parameters of the SIR model. (They also show up in the stochastic differential equations of the model.) $\beta$ is governing the rate at which infections take place, whereas $\gamma$ determines how fast people recover.

Now this looks pretty much like the conditional intensity of a Hawkes process with exponential kernel! To see this more easily, just set $\mu = 0$,
\begin{align}
\kappa \theta = \beta,
\end{align}
and
\begin{align}
\theta = \gamma.
\end{align}
Now the only difference is the factor in front of the sum!

To eliminate this last difference between the two models, Rizoiu et al. defined the HawkesN process. This process is defined by its conditional intensity, which is the same as that of a Hawkes process, but without the background intensity $\mu(t)$ and with the additional factor $(1-N_t/N)$, where $N_t$ denotes the HawkesN process at time $t$. So compared to a Hawkes process, the HawkesN process lacks the parameter $\mu(t)$, but features the additional parameter $N$, which can be interpreted as the population's size.

This additional factor in the conditional intensity – let's call it HawkesN factor – has an important effect. As is the case with the Hawkes process, infections tend to increase the conditional intensity of the HawkesN process. However, as the number of events rises, this increasing effect declines for the HawkesN process and eventually, further events can even have a decreasing effect, since the factor in front of the sum decreases.

With diseases we make similar observations. At first, infections increase the probability of further infections. However, as more and more people become infected, the number of susceptibles declines and this lowers the probability of further infections, as there are fewer people left who can become infected.

Using the link found by Rizoiu et al. we can translate an SIR model easily into a HawkesN process and vice versa using the equations above. This allows one for example to estimate model parameters in one model and then make predictions using the other model.

### The SEIR Model

The SEIR model is a generalization of the SIR model, adding a compartment between S and I. This compartment is called E, which stands for *exposed*. Members of this group are infected, but are not able to infect others yet. After some time, people move on to compartment I, which means that they become infectious. The states and possible transitions are summarized in the following figure.

![SEIR model](./fig/flowcharts/seir.svg)

The additional compartment allows for modeling incubation time – a key characteristic of many infectious diseases.{% fn 5 %}

{{ 'The incubation time – that is the time a persion spends in compartment E – depends on the model parameter $\sigma$. The lower the value of $\sigma$, the longer the incubation period. You can see this effect in the interactive chart at the top of this blog post; just move the slider for $\sigma$ and see what happens.' | fndetail: 5 }}

We have found that the SEIR model also has a close relationship to the HawkesN process. However, for the expectation of the conditional intensity $\lambda_t^{SEIR}$ to match that of a HawkesN process, the kernel of the HawkesN process must not be the exponential kernel, but rather the *hypoexponential* kernel or the *Erlang* kernel. The conditional intensity with a hypoexponential kernel looks like
\begin{align}
\lambda_t
= \left(
    1 - \frac{N_t}{N}
  \right)
  \sum_{t_j < t}
    \kappa \frac{\theta \sigma}{\theta - \sigma}
    \left( 
      \text{e}^{-\sigma (t-t_j)} - \text{e}^{-\theta (t-t_j)}
    \right),
\end{align}
where everything that follows the scale parameter $\kappa$ is the density of a hypoexponential distribution.

You may notice that this formula doesn't work for the case $\sigma = \theta$, as we would divide by zero. We have shown that in this case the link between SEIR and HawkesN is established with the Erlang kernel, which leads to the conditional intensity
\begin{align}
\lambda_t
= \left(
    1 - \frac{N_t}{N}
  \right)
  \sum_{t_j < t}
    \kappa \theta^2
    (t-t_j)
    \text{e}^{-\theta (t-t_j)}
.
\end{align}

How do these formulas differ from the conditional intensity with an exponential kernel? Have a look at the following chart. You can select a sequence of event times below the chart and the plot will update accordingly. To highlight the conditional intensity with a specific kernel, just klick on the corresponding label in the legend. (To select multiple kernels, hold the shift key while klicking.)

In [53]:
#hide_input
event_seq_ids = ["{0}",
                 "{0, 1, 2}",
                 "{0, 1, 2, ..., 9}"]

lambda_t_unicode = "\u03BBₜ"

url_conditional_intensities = "https://raw.githubusercontent.com/yogabonito/blog/master/_notebooks/fig/conditional_intensities.json"

selection_kernel = alt.selection_multi(fields=['kernel'], bind='legend')

event_seq_radio = alt.binding_radio(options=event_seq_ids,
                                    name="Event times: ")
event_seq_select = alt.selection_single(fields=["event sequence"],
                                        bind=event_seq_radio,
                                        name="event Sequence",
                                        init={"event sequence": event_seq_ids[1]})

alt.Chart(url_conditional_intensities).mark_line().encode(
    x="t:Q",
    #y=lambda_t_unicode+":Q",
    y=alt.Y(
        lambda_t_unicode + ":Q",
        scale=alt.Scale(domain=(0, 3.5))
    ),
    color="kernel:N",
    tooltip=["t:Q", lambda_t_unicode + ":Q", "kernel:N"],
    opacity=alt.condition(selection_kernel, alt.value(1), alt.value(0.2)),
).add_selection(
    selection_kernel
).add_selection(
    event_seq_select
).transform_filter(
    event_seq_select
)

alt.Chart(...)

We can see that a high value of the parameter $\sigma$ leads to a conditional intensity that resembles that with an exponential kernel. That's because the hypoexponential kernel converges to the exponential one – a property inherited from the densities of the hypoexponential and the exponential distribution:
\begin{align}
\lim_{\sigma \to \infty}
\frac{\theta \sigma}{\theta - \sigma}
    \left( 
      \text{e}^{-\sigma (t-t_j)} - \text{e}^{-\theta (t-t_j)}
    \right)
= \theta \text{e}^{-\theta (t - t_j)}.
\end{align}


The same relationship exists between the SEIR model and the SIR model. When $\sigma$ goes to infinity, the time spent in compartment E goes to zero, thus eliminating the need for this compartment.

Now, let's have a look at smaller values of $\sigma$. They lead to a large delay with which the conditional intensity increases. In the SEIR model a lower $\sigma$ means that people spend more time in compartment E where they are not yet able to infect others. Thus, an infection doesn't immediately increase the probability of further infections but only does so with some delay.

If we select the event sequence $\{0, 1, 2, \ldots, 9\}$ below the chart, we can see that the conditional intensity drops to zero at $t=9$. This is because in this example we assumed a population of only ten people. At $t=9$, the $10^{\text{th}}$ event occurs, so the factor $(1 - 10 / 10)$ in the conditional intensity of a HawkesN process becomes zero.

In an S(E)IR model this $10^{\text{th}}$ infection event has the same effect. As soon as everybody left the group of susceptibles, there is nobody left who can get infected.

The event sequence with ten events nicely reflects how infections can have an increasing effect on the conditional intensity at first. However, we can also see that the last two events decrease $\lambda_t$ – just as discussed above when we defined the HawkesN factor. Note that this decreasing effect eventually prevails irrespective of the kernel function used.

## Summary

In this blog post we have briefly presented the link between the SEIR model and Hawkes processes, which can be seen as a generalization of Rizoiu et al.'s work. Further generalizations are possible and we believe that they can proof valuable for researchers utilizing extensions to the basic compartmental epidemic models.

## References

{{ 'Hawkes (1971): "Spectra of some self-exciting and mutually exciting point processes"' | fndetail: 1 }}

{{ 'Rizoiu et al. (2018): "SIR-Hawkes: Linking Epidemic Models and Hawkes Processes to Model Diffusions in Finite Populations"' | fndetail: 2 }}

{{ 'https://todo' | fndetail: 3 }}

{{ 'https://mathworld.wolfram.com/HawkesProcess.html' | fndetail: 4 }}